In [177]:
import bibtexparser as bp
import pandas as pd
import yaml as yml

# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dirConfig = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos'
configFileName = 'config_analisa_bibtex.yml'

with open(dirConfig + '\\' + configFileName) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)


In [178]:
# ANALISA BIBTEX

def funcAnalisaBibtex(FILE_ACM, FILE_IEE, FILE_SD, FORMATO):

	acmFile = FILE_ACM
	ieeeFile = FILE_IEE
	sdFile = FILE_SD
	formatFile = FORMATO
	
	dirInput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\input'
	dirOutput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\output'

	mergedFilesCSV = 'ALL_ARTICLES.csv'
	mergedFilesJSON = 'ALL_ARTICLES.json'
	mergedFilesYAML = 'ALL_ARTICLES.yml'
	
	with open(dirInput + '\\' + acmFile, encoding='utf8') as acmf, \
	     open(dirInput + '\\' + ieeeFile, encoding='utf8') as ieeef, \
		 open(dirInput + '\\' + sdFile, encoding='utf8') as sdf:
	
		bib_database_acm = bp.load(acmf)
		bib_database_iee = bp.load(ieeef)
		bib_database_sd  = bp.load(sdf)
	
	dfAcm  = pd.DataFrame(bib_database_acm.entries_dict.values()) 
	dfIEEE = pd.DataFrame(bib_database_iee.entries_dict.values()) 
	dfSD   = pd.DataFrame(bib_database_sd.entries_dict.values()) 
	
	# COMPARA COLUNAS
	setAcm, setIEEE, setSD = set(dfAcm.columns), set(dfIEEE.columns), set(dfSD.columns)
	selColunas = setAcm.intersection(setIEEE,setSD)
	
	# UNION DFs
	dfUnion = pd.concat([dfAcm.get(selColunas), dfIEEE.get(selColunas), dfSD.get(selColunas)])
	
	# EXPORT
	if formatFile == 'CSV':
		dfUnion.to_csv(dirOutput + '\\' + mergedFilesCSV, index=False, encoding='utf-8')
	elif formatFile == 'JSON':
		dfUnion.to_json(dirOutput + '\\' + mergedFilesJSON, orient = 'records')
	elif formatFile == 'YAML':	
		ymlFileACM = yml.dump(list(bib_database_acm.entries_dict.values()))
		ymlFileIEEE = yml.dump(list(bib_database_iee.entries_dict.values()))
		ymlFileACM = yml.dump(list(bib_database_sd.entries_dict.values()))

		ymlUnion = ymlFileACM + ymlFileIEEE + ymlFileACM
		ymlUnion

		with open(dirOutput + '\\' + mergedFilesYAML, mode='w',encoding='utf8') as f:
			f.write(ymlUnion)
	else:
		print('Formato não disponivel.')

In [179]:
funcAnalisaBibtex(\
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ], \
	configFile['FORMATO' ]
	)